In [ ]:
import ipywidgets as widgets

button = widgets.Button(description="Run!", button_style="success")
box = widgets.VBox(children=[button])
box

In [ ]:
from IPython.display import clear_output
from fake_api import search_restaurants, visited_restaurants
import pandas as pd


def query_and_enrich():
    "Query for restaurants and enrich with history, returns dataframe"
    df = pd.DataFrame(search_restaurants())
    df = df.T

    df2 = pd.DataFrame([visited_restaurants()])
    df2 = df2.T
    df2.columns = ["history"]
    df2["last_visited"] = df2.history.apply(lambda lst: max(lst))

    df = df.join(df2[["last_visited"]])
    return df


### To hold the list of restaurants:
dropdown = widgets.Dropdown()

### To print out info about a restaurant
output = widgets.Output()

### To change the list of restaurants:
toggle = widgets.ToggleButtons(
    options=["All", "Visited", "Unvisited"],
    description="Filter:",
    disabled=False,
    button_style="",  # 'success', 'info', 'warning', 'danger' or ''
    tooltips=["All restaurants", "Previously visited", "Never visited"],
)


def on_toggle(ev):
    if toggle.value == "All":
        frame = dropdown.df
    elif toggle.value == "Visited":
        frame = dropdown.df[dropdown.df.last_visited.notnull()]
    else:
        frame = dropdown.df[dropdown.df.last_visited.isnull()]
    dropdown.options = [" "] + sorted(frame.index)


toggle.observe(on_toggle, names="value")


def on_select(ev):
    with output:
        clear_output()
        item = dropdown.value
        if item.strip():
            row = dropdown.df.loc[item]
            print("Name: %s" % row.name)
            print("Price: %s" % row["price"])
            print("Rating: %s" % row["rating"])


dropdown.observe(on_select, names="value")


def run(ev):
    "Handler to run everything when button is clicked"
    button.disabled = True
    button.button_style = "warning"
    df = query_and_enrich()
    dropdown.df = df
    dropdown.options = [" "] + sorted(df.index)
    box.children = [dropdown, toggle, output]


button.on_click(run)